In [1]:
#| include: false

import os
import pandas as pd
import numpy as np
import toml
from pathlib import Path
import summary_data

config = summary_data.CONFIG
all_runs = summary_data.ALL_RUNS

In [2]:
# all_runs

- run locations

In [3]:
pd.DataFrame.from_dict(all_runs, orient='index',
                       columns=['run location'])

,run location
current run,\\modelstation4\c$\workspace\sc_2050_old_trans...


## System Summary

In [4]:
# network summary: 'VMT','VHT','total_delay'
df_network = summary_data.load_agg_data('network/network_results.csv')

C:\workspace\sc_organize_summary_folder\scripts\summarize\summary\summary_scripts\summary_data.py:86: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df["facility_type"].fillna("Other", inplace=True)
C:\workspace\sc_organize_summary_folder\scripts\summarize\summary\summary_scripts\summary_data.py:91: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate o

In [5]:
# Get light rail boardings
df_boardings = summary_data.load_agg_data('transit/daily_boardings_by_agency.csv')

In [6]:
network_summary = df_network.groupby('source')[['VMT','VHT','total_delay']].sum()
network_summary = network_summary.rename(columns={'total_delay': 'Delay'})
# transit boardings
df_boardings = summary_data.load_agg_data('transit/daily_boardings_by_agency.csv')
transit_ridership = df_boardings.groupby('source')['boardings'].sum()
#Light rail boardings
df_line_boardings = summary_data.load_agg_data('transit/transit_line_results.csv')
lr_boardings = df_line_boardings[df_line_boardings['mode']=='r']
lr_ridership = lr_boardings.groupby('source')['boardings'].sum()
# mode share
df_trip = summary_data.load_agg_data('agg/dash/mode_share_county.csv')
df = df_trip.groupby('source')['trexpfac'].sum() # total trips
transit_share = df_trip.loc[df_trip['mode']=="Transit"].groupby('source')['trexpfac'].sum()/df

# emission
df_emissions = summary_data.load_agg_data('emissions/emissions_summary.csv')
df = df_emissions.loc[(df_emissions['veh_type'].isin(['light','medium','heavy'])) & \
                      (df_emissions['pollutant_name']=="CO2 Equivalent")].copy()
CO2e = df.groupby('source')['total_daily_tons'].sum()

In [7]:
run_summary2 = pd.DataFrame({
    'Transit Boardings': transit_ridership,
    'Light Rail Boardings': lr_ridership,
    '% Transit': transit_share,
    'CO2e': CO2e
})

pd.concat([network_summary,run_summary2], axis=1).style.\
        format('{:,.0f}', subset=['VMT','VHT','Delay','Transit Boardings','Light Rail Boardings','CO2e']).\
        format('{:.1%}', subset=['% Transit'])

,VMT,VHT,Delay,Transit Boardings,Light Rail Boardings,% Transit,CO2e
source,,,,,,,
current run,"95,751,281","3,013,369","282,649","1,569,048","473,300",4.8%,"6,393"
